#  SGmarkets ROTB strategy prices endpoint

**The [compute-strategy/prices endpoint](https://analytics-api.sgmarkets.com/rotb/v1/swagger/ui/index#!/Swaption/Swaption_StrategyStrategy2) enables to compute the historical performances of vol strategies**.   
**The calculation can be performed using either relative (analytics) or absolute (backtests) parameters.** 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

import datetime as dt
import numpy as np
import pandas as pd

from pandas.tseries.offsets import BDay

from sgmarkets_api_auth import Api
from sgmarkets_plot import Plot
import sgmarkets_api_analytics_rotb as ROTB

from sgmarkets_api_auth.util import topickle, unpickle
import sgmarkets_api_analytics_rotb.biz as biz

<IPython.core.display.Javascript object>

# A - Relative parameters (analytics)

## 1 - Authenticate

In [3]:
a = Api(verbose=True)

Variables in /Users/Olivier/my_secret.txt
SG_LOGIN=o*******m
SG_PASSWORD=m************************d
PROXY_LOGIN=m*******n
PROXY_PASSWORD=m***********************************d
PROXY_HOST=m*******************t
PROXY_PORT=9******9
Token endpoint: https://login.sgmarkets.com/richclient/gettoken
proxies: {}
Access token {b102****************************532e} loaded from file /Users/Olivier/my_token.txt


## 2 - Build Request
### Example: 10Y JPY 3M6M ATMF reverse receiver calendar spread

In [4]:
ep = ROTB.endpoint.v1_compute_strategy_prices
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=1)

rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'DV01'

rq.curve = 'JPY LIBOR 6M' 
rq.expiry = '3M,6M'
rq.tenor = '10Y'
rq.strike= '@0bp'
rq.type = 'receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'Auto'
rq.nominal = int(1e6)
rq.strategyWeight = '1,-1'

rq.expand()
rq.info()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 6 dates i.e. total 12 prices.


A RequestROTB object has the properties after the `expand()` method:
+ `url`: https://analytics-api.sgmarkets.com/rotb/v1/compute-strategy/prices
+ `dic_input`: user input (dictionary)
+ `df_top`: parameters of the request not in a leg (dataframe)
+ `df_leg`: parameters of the request in a leg (dataframe)
    + A leg contains the following parameters: ['curve', 'expiry', 'tenor', 'strike', 'nominal', 'type', 'settlement', 'pricingStrategy', 'strategyWeight', 'customEntries']
    + Each param can have multiple comma separated values
+ `li_dic_api`: data ready to be sent to the API - built from user input by `expand()` (list of dictionaries)
    + This is a list because the request may be split in several chunks
    + Legs are constructed from user input and by filling missing values by repition if needed.
        

## 3.1 - Make Request
Save response to work locally.

In [5]:
res1 = rq.call_api(a)

topickle('101.pk', res1)

calling API...
chunk 1/1 done in 1.08 s


## 3.2 - Response Structure 

In [6]:
res1 = unpickle('101.pk')
res1.info()


A PostprocessROTB object from ComputeStrategyPrices endpoint has the properties:
+ `df_req`: request data (dataframe)
+ `df_res`: response data (dataframe)


+ `dic_req_param`: params in request, each param contains a list of all values taken (dictionary)
+ `dic_res_param`: params in response, each param contains a list of all values taken (dictionary)

+ `raw_data`: raw data in response under key 'componentSeries' (dictionary)

and the methods:
+ `save()` to save the data as `.csv` and `.xlsx` files
        

## 4 - Show Response

In [7]:
res1.df_res

,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,5.684342e-14,-1630.448170,20.124458,-1631.092692,-3.214910,-81.699550,-0.000167,0
2018-03-27,1.136868e-13,-1512.345992,18.681607,-1512.968577,-3.651218,-81.641330,-0.000118,1
2018-03-28,1.136868e-13,-1492.327326,18.091668,-1492.959098,-3.815134,-81.636189,-0.000105,2
2018-03-29,0.000000e+00,-1591.261614,20.321986,-1591.900695,-3.130205,-80.857121,-0.000164,3
2018-03-30,5.684342e-14,-1565.245672,20.958722,-1565.871429,-3.109203,-81.147695,-0.000162,4
2018-04-02,0.000000e+00,-1545.532586,20.871146,-1546.153269,-3.213522,-81.943458,-0.000151,5


## 5 - Slice Response 
The response is a pandas dataframe.  
Use the built in methods.

In [8]:
display(res1.df_res[['forwardValue']])
display(pd.DataFrame(res1.df_res.loc['2018-03-26']).T)

,forwardValue
date,
2018-03-26,-1630.448170
2018-03-27,-1512.345992
2018-03-28,-1492.327326
2018-03-29,-1591.261614
2018-03-30,-1565.245672
2018-04-02,-1545.532586


,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
2018-03-26,5.684342e-14,-1630.44817,20.124458,-1631.092692,-3.21491,-81.69955,-0.000167,0.0


## 6 - Example #2
### Straddle strategy (EUR 3M10Y ATMF straddle)

In [9]:
ep = ROTB.endpoint.v1_compute_strategy_prices
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=1)
rq.targetCurrency='EUR' # define the price currency 
rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'DV01'

rq.curve = 'EUR EURIBOR 6M' 
rq.expiry = '3M'
rq.tenor = '10Y'
rq.strike= '@0bp'
rq.type = 'payer,receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'Auto'
rq.nominal = int(1e6)
rq.strategyWeight = '1'

rq.expand()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 6 dates i.e. total 12 prices.

In [10]:
res1 = rq.call_api(a)
res1.df_res

calling API...
chunk 1/1 done in 0.81 s


,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,9.453089e-07,14248.222628,39.890718,14261.188354,-77.506458,378.670321,0.007532,0
2018-03-27,9.462721e-07,14144.286325,40.265788,14157.150327,-76.941034,379.056112,0.007470,1
2018-03-28,9.469999e-07,14319.673681,39.833846,14332.693578,-77.895074,379.347746,0.007556,2
2018-03-29,9.468211e-07,13978.502827,40.790789,13991.163105,-76.038930,379.276070,0.007378,3
2018-03-30,9.472560e-07,13970.302651,40.852635,13982.819085,-76.828676,377.382425,0.007410,4
2018-04-02,9.474223e-07,13871.590696,41.157804,13884.017462,-76.285810,377.448702,0.007357,5


## 7 - Example #3
### Strangle strategy (EUR 3M10Y 25 delta strangle)

In [11]:
# delta 25 on both side
ep = ROTB.endpoint.v1_compute_strategy_prices
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=1)
rq.targetCurrency='EUR'
rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'None'

rq.curve = 'EUR EURIBOR 6M' 
rq.expiry = '3M'
rq.tenor = '10Y'
rq.strike= 'd25%'
rq.type = 'payer,receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'GaussShiftedGauss'
rq.nominal = int(1e6)
rq.strategyWeight = '1'

rq.expand()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 6 dates i.e. total 12 prices.

In [12]:
res1 = rq.call_api(a)

calling API...
chunk 1/1 done in 0.98 s


In [13]:
res1.df_res

,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,7.062619,5448.318283,31.352756,5453.276192,-63.085536,301.541796,0.007693,0
2018-03-27,7.025587,5386.919462,31.742713,5391.818779,-62.400120,301.768497,0.007604,1
2018-03-28,5.986941,5446.944074,31.371526,5451.896607,-63.104749,302.044197,0.007684,2
2018-03-29,6.446252,5315.264233,32.159252,5320.078248,-61.581267,301.948479,0.007501,3
2018-03-30,6.371922,5302.138685,32.250079,5306.889038,-62.115586,300.407153,0.007522,4
2018-04-02,6.105670,5264.867397,32.474872,5269.583891,-61.678814,300.478393,0.007468,5


## 8 - Example #4
### Risk-reversal (EUR 3M10Y 25 delta risk reversal)

In [14]:
# delta 25 on both side
ep = ROTB.endpoint.v1_compute_strategy_prices
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=1)
rq.targetCurrency='EUR'
rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'None'

rq.curve = 'EUR EURIBOR 6M' 
rq.expiry = '3M'
rq.tenor = '10Y'
rq.strike= 'd25%'
rq.type = 'payer,receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'GaussShiftedGauss'
rq.nominal = int(1e6) 
rq.strategyWeight = '1,-1'

rq.expand()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 6 dates i.e. total 12 prices.

In [15]:
res1 = rq.call_api(a)

calling API...
chunk 1/1 done in 0.96 s


In [16]:
res1.df_res

,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,472.458418,616.334170,-2.688553,616.895027,-6.402118,2.392811,0.000720,0
2018-03-27,472.701439,606.888575,-2.712155,607.440530,-6.307978,2.381662,0.000709,1
2018-03-28,473.176806,524.453627,-2.291753,524.930477,-5.452357,2.028992,0.000613,2
2018-03-29,472.980429,550.203349,-2.525037,550.701666,-5.719950,2.185235,0.000643,3
2018-03-30,473.096350,542.706760,-2.504163,543.192989,-5.705406,2.148801,0.000638,4
2018-04-02,473.230139,516.646661,-2.417651,517.109496,-5.431407,2.058753,0.000607,5


## 9 - Example #5
### Triangle (EUR 1y10y 1y30y 11y20y ATMF straddle triangle)

In [17]:
# delta 25 on both side
ep = ROTB.endpoint.v1_compute_strategy_prices
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=1)
rq.targetCurrency='EUR'
rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'None'

rq.curve = 'EUR EURIBOR 6M' 
rq.expiry = '1Y, 1Y,1Y,1Y,11Y,11Y'
rq.tenor = '10Y,10Y,30Y,30Y,20Y,20Y'
rq.strike= '@0bp'
rq.type = 'payer,receiver,payer,receiver,payer,receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'Auto'
rq.nominal = int(1e6/2) 
rq.strategyWeight = '1,1,-1,-1,1,1'

rq.expand()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 6 prices over 6 dates i.e. total 36 prices.

In [18]:
res1 = rq.call_api(a)
display(res1.df_res)

calling API...
chunk 1/1 done in 1.18 s


,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,3.159533e-08,93253.035405,-9.531536,81933.167901,21.940659,1420.892665,0.011027,0
2018-03-27,3.177894e-08,95593.919813,-9.755294,84271.110695,21.428260,1426.030184,0.011185,1
2018-03-28,3.205065e-08,95899.288840,-9.653773,84690.013171,21.796222,1429.929603,0.011207,2
2018-03-29,3.271458e-08,91691.408406,-9.529598,80862.789027,22.077030,1426.627475,0.010838,3
2018-03-30,3.294213e-08,92230.924782,-9.548051,81443.005734,22.122381,1428.846852,0.010921,4
2018-04-02,3.300096e-08,92335.165025,-9.512582,81529.106662,22.122339,1428.145799,0.010879,5


# B - Absolute parameters (backtests)

## 1 - Build Request
### Example: 10Y JPY 3M6M ATMF reverse receiver calendar spread)

In [19]:
from dateutil.relativedelta import relativedelta

ep = ROTB.endpoint.v1_compute_strategy_prices
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=1)
expiry_1=start+relativedelta(months=+3) # first expiry date
expiry_2=start+relativedelta(months=+6) # second expiry date
tenor_1=expiry_1+relativedelta(years=+10) # first tenor date
tenor_2=expiry_2+relativedelta(years=+10)# second tenor date

expiry_1=expiry_1.strftime('%Y-%m-%d')
expiry_2=expiry_2.strftime('%Y-%m-%d')
tenor_1=tenor_1.strftime('%Y-%m-%d')
tenor_2=tenor_2.strftime('%Y-%m-%d')

rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.targetCurrency='EUR'
rq.weighting = 'DV01'

rq.curve = 'JPY LIBOR 6M' 
rq.expiry = [expiry_1,expiry_2]
rq.tenor = [tenor_1,tenor_2]
rq.strike= '0.3254%, 0.345%'
rq.type = 'receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'Auto'
rq.nominal = int(1e6)
rq.strategyWeight = '1,-1'

rq.expand()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 6 dates i.e. total 12 prices.

In [20]:
res1 = rq.call_api(a)
res1.df_res

calling API...
chunk 1/1 done in 1.00 s


,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,-46.781315,-1801.610454,16.167383,-1802.474940,-2.726120,-86.103384,-0.000184,0
2018-03-27,-25.076551,-1667.225094,17.725371,-1667.972433,-3.526498,-82.896925,-0.000139,1
2018-03-28,-22.678780,-1626.265024,17.421837,-1627.010560,-3.817712,-82.951130,-0.000119,2
2018-03-29,-17.666706,-1698.158194,20.077581,-1698.879216,-3.348036,-82.815449,-0.000168,3
2018-03-30,-6.916818,-1672.223129,21.559867,-1672.897671,-3.302395,-82.553050,-0.000169,4
2018-04-02,-9.199553,-1663.061128,22.067703,-1663.731070,-3.509944,-83.642693,-0.000162,5


## 2 - Example #2
### Straddle (ex: EUR 3M10Y ATMF straddle)

In [21]:
ep = ROTB.endpoint.v1_compute_strategy_prices
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=1)
expiry=start+relativedelta(months=+3)
tenor=expiry+relativedelta(years=10)
expiry=expiry.strftime('%Y-%m-%d')
tenor=tenor.strftime('%Y-%m-%d')

rq.targetCurrency='EUR' # define the price currency 
rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'DV01'

rq.curve = 'EUR EURIBOR 6M' 
rq.expiry = expiry
rq.tenor = tenor
rq.strike= '1.1232%'
rq.type = 'payer,receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'Auto'
rq.nominal = int(1e6)
rq.strategyWeight = '1'

rq.expand()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 6 dates i.e. total 12 prices.

In [22]:
res1 = rq.call_api(a)
res1.df_res

calling API...
chunk 1/1 done in 0.80 s


,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,-261.373445,15893.794966,35.693255,15908.101562,-77.321991,353.878801,0.007953,0
2018-03-27,-329.739528,16492.747985,34.333511,16507.423339,-75.277644,338.690640,0.008001,1
2018-03-28,-378.826624,17136.532161,32.933802,17151.609569,-74.038506,325.318179,0.008102,2
2018-03-29,-379.851921,16712.806550,33.755316,16727.290315,-72.911841,323.162799,0.007942,3
2018-03-30,-407.876423,17109.628544,32.872395,17124.289014,-72.332146,314.028574,0.008016,4
2018-04-02,-416.330989,16748.890782,33.530635,16762.749927,-72.327100,306.209925,0.007936,5


## 3 - Example #3
### Strangle (ex:  EUR 3M10Y ATMF+/-25delta strangle)

In [23]:
# delta 25 on both side
ep = ROTB.endpoint.v1_compute_strategy_prices
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=1)
expiry=start+relativedelta(months=+3)
tenor=expiry+relativedelta(years=10)
expiry=expiry.strftime('%Y-%m-%d')
tenor=tenor.strftime('%Y-%m-%d')

rq.targetCurrency='EUR'
rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'None'

rq.curve = 'EUR EURIBOR 6M' 
rq.expiry = expiry
rq.tenor = tenor
rq.strike= '1.2746%,0.9903%'
rq.type = 'payer,receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'GaussShiftedGauss'
rq.nominal = int(1e6)
rq.strategyWeight = '1'

rq.expand()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 6 dates i.e. total 12 prices.

In [24]:
res1 = rq.call_api(a)
res1.df_res

calling API...
chunk 1/1 done in 0.80 s


,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,-191.464836,6028.834349,30.054442,6034.261128,-63.576031,292.764359,0.008091,0
2018-03-27,-250.887152,6434.692638,29.858499,6440.418269,-62.722013,286.748477,0.008111,1
2018-03-28,-298.322677,6971.756114,29.123681,6977.890145,-62.709796,280.379624,0.008211,2
2018-03-29,-293.904974,6594.515650,29.789040,6600.230634,-61.197697,276.822259,0.008046,3
2018-03-30,-320.161724,6897.571908,29.447921,6903.482127,-61.280688,272.368842,0.008109,4
2018-04-02,-323.789629,6606.398137,29.867561,6611.864710,-60.881378,264.038696,0.008033,5
